In [ ]:
import tensorflow as tf

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip

In [ ]:
!unzip DIV2K_train_HR.zip

In [ ]:
import glob,os
img_dir = "DIV2K_train_HR"
data_path = os.path.join(img_dir,"*g")
files = glob.glob(data_path)

In [ ]:
len(files)

In [ ]:
import cv2
import numpy as np

def loadLRImages(imagelist):
    images=[]
    for image in (imagelist):
        img = cv2.resize(cv2.GaussianBlur(cv2.imread(image),(5,5),cv2.BORDER_DEFAULT),(64,64)) 

        images.append(img)
    return np.array(images)

In [ ]:
def loadHRImages(imagelist):
    images=[]
    for image in (imagelist):
        img = cv2.resize(cv2.imread(image),(256,256))
        images.append(img)
    return np.array(images)

In [ ]:
images_LR = loadLRImages(files)

In [ ]:
images_HR = loadHRImages(files)

In [ ]:
def random_crop(lr_img, hr_img, hr_crop_size=96, scale=2):
    lr_crop_size = hr_crop_size // scale
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w * scale
    hr_h = lr_h * scale

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped

In [ ]:
cropped_lr_images = []
cropped_hr_images = []
for i in range(800):
  for j in range(20):
    lr_img,hr_img = random_crop(images_LR[j],images_HR[j])
  
    cropped_lr_images.append(lr_img)
    cropped_hr_images.append(hr_img)



In [ ]:
np.asarray(cropped_hr_images).shape

In [ ]:
def B_res_blocks(x,n_filters = 64,training = True):
  x_3 = tf.keras.layers.Conv2D(filters=n_filters,kernel_size=(3,3),strides=1,padding = 'same')(x)
  if training:
    x_3 = tf.keras.layers.BatchNormalization()(x_3)
  x_4 = tf.keras.layers.PReLU()(x_3)
  x_5 = tf.keras.layers.Conv2D(filters = n_filters,kernel_size=(3,3),strides=1,padding = 'same')(x_4)
  if training:
    x_5 = tf.keras.layers.BatchNormalization()(x_5)
  
  out = tf.keras.layers.Add()([x,x_5])
  return out

In [ ]:
def gen_sub(input_shape, n_filters = 64,training = True):
  input_image = tf.keras.Input(shape = input_shape)
  c1 = tf.keras.layers.Conv2D(filters=n_filters,kernel_size=(9,9),strides=1,padding = 'same')(input_image)
  c1 = tf.keras.layers.PReLU()(c1)
  c2 = B_res_blocks(c1,training)
  c3 = B_res_blocks(c2,training)
  c4 = B_res_blocks(c3,training)
  c5 = B_res_blocks(c4,training)
  c6 = B_res_blocks(c5,training)
  c7 = tf.keras.layers.Conv2D(filters=n_filters,kernel_size=(9,9),strides=1,padding= 'same')(c6)
  if training:
    c7 = tf.keras.layers.BatchNormalization()(c7)
  c8 = tf.keras.layers.Add()([c1,c7])
  c9 = tf.keras.layers.Conv2D(filters=n_filters*4,kernel_size=(3,3),strides=1,padding = 'same')(c8)
  c10 = tf.keras.layers.UpSampling2D(size=(2,2))(c9)
  c11 = tf.keras.layers.PReLU()(c10)
  c12 = tf.keras.layers.Conv2D(filters=n_filters*4,kernel_size=(3,3),strides=1,padding = 'same')(c11)
  c13 = tf.keras.layers.UpSampling2D(size=(2,2))(c12)
  c14 = tf.keras.layers.PReLU()(c13)
  out = tf.keras.layers.Conv2D(filters = 3,kernel_size=(9,9),strides=1,padding = 'same')(c14)
  gen = tf.keras.Model(input_image,out)
  return gen

In [ ]:
def disc_sub(input_shape,n_filters = 64,training = True):
  input_image = tf.keras.Input(shape = input_shape)
  u1 = tf.keras.layers.Conv2D(filters = n_filters,kernel_size = (3,3),strides=1,padding = 'same')(input_image)
  u2 = tf.keras.layers.LeakyReLU()(u1)
  u3 = tf.keras.layers.Conv2D(filters = n_filters,kernel_size = (3,3),strides=2,padding = 'same')(u2)
  if training:
    u3 = tf.keras.layers.BatchNormalization()(u3)
  u4 = tf.keras.layers.LeakyReLU()(u3)
  u4 = tf.keras.layers.Conv2D(filters = n_filters*2,kernel_size = (3,3),strides=1,padding = 'same')(u4)
  if training:
    u4 = tf.keras.layers.BatchNormalization()(u4)
  u5 = tf.keras.layers.LeakyReLU()(u4)
  u5 = tf.keras.layers.Conv2D(filters = n_filters*2,kernel_size = (3,3),strides=2,padding = 'same')(u5)
  if training:
    u5 = tf.keras.layers.BatchNormalization()(u5)
  u6 = tf.keras.layers.LeakyReLU()(u5)
  u6 = tf.keras.layers.Conv2D(filters = n_filters*4,kernel_size = (3,3),strides=1,padding = 'same')(u6)
  if training:
    u6 = tf.keras.layers.BatchNormalization()(u6)
  u7 = tf.keras.layers.LeakyReLU()(u6)
  u7 = tf.keras.layers.Conv2D(filters = n_filters*4,kernel_size = (3,3),strides=2,padding = 'same')(u7)
  if training:
    u7 = tf.keras.layers.BatchNormalization()(u7)
  u8 = tf.keras.layers.LeakyReLU()(u7)
  u8 = tf.keras.layers.Conv2D(filters = n_filters*8,kernel_size = (3,3),strides=1,padding = 'same')(u8)
  if training:
    u8 = tf.keras.layers.BatchNormalization()(u8)  
  u9 = tf.keras.layers.LeakyReLU()(u8)
  u9 = tf.keras.layers.Conv2D(filters = n_filters*8,kernel_size = (3,3),strides=2,padding = 'same')(u9)
  if training:
    u9 = tf.keras.layers.BatchNormalization()(u9)  
  u10 = tf.keras.layers.LeakyReLU()(u9) 
  u10 = tf.keras.layers.Flatten()(u10)
  u10 = tf.keras.layers.Dense(1024)(u10)
  u10 = tf.keras.layers.LeakyReLU()(u10)
  out = tf.keras.layers.Dense(1,activation="sigmoid")(u10)
  disc = tf.keras.Model(input_image,out)
  return disc

In [ ]:
train_x = np.asarray(cropped_lr_images[12000:15200])
train_y = np.asarray(cropped_hr_images[12000:15200])
test_x = np.asarray(cropped_lr_images[15200:16000])
test_y = np.asarray(cropped_hr_images[15200:16000])


In [ ]:
len(cropped_hr_images)

In [ ]:
genet = gen_sub(train_x.shape[1:])
disc_hr = disc_sub(train_y.shape[1:])
disc_sr = disc_sub((192,192,3))
disc_hr.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.002),metrics=['accuracy'])
disc_sr.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.002),metrics=['accuracy'])

frozen = tf.keras.Model(disc_sr.inputs,disc_sr.outputs)
frozen.trainable = False
noise = tf.keras.Input(shape = (48,48,3))
image = genet(noise)
logit = frozen(image)
srgan = tf.keras.Model(noise,logit)

In [ ]:
srgan.compile(loss =, optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.002),metrics=['accuracy'])

In [ ]:
srgan.compile(loss =tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.002),metrics=['accuracy'])

In [ ]:
train_x.shape

In [ ]:

batch_size = 10
n_critic = train_x.shape[0]//batch_size

In [ ]:
srgan = tf.keras.models.load_model("srgan_model")
genet = tf.keras.models.load_model("gen_sub_model")
disc_hr = tf.keras.models.load_model("disc_hr_model")
disc_sr = tf.keras.models.load_model("disc_sr_model")

In [ ]:

clip_value = 0.01
epochs = 10
for epoch in range(epochs):
  for i in range(n_critic):
    batch_lr = train_x[i*batch_size:(i+1)*batch_size]
    batch_hr = train_y[i*batch_size:(i+1)*batch_size]
    
    batch_sr = genet.predict(batch_lr)

    d_loss_real = disc_hr.train_on_batch(batch_hr,np.ones((batch_size,1)))
    d_loss_fake = disc_sr.train_on_batch(batch_sr,np.zeros((batch_size,1)))
    d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
    filepath = "checkpoint"
    disc_hr.trainable = False
    disc_sr.trainable = False
    print(batch_lr.shape)
    g_loss = srgan.train_on_batch(batch_lr,-np.ones((batch_size,1)))
    disc_hr.trainable = True
    disc_sr.trainable = True    
    
  print("epoch {} and loss {} ".format(epoch,g_loss[0]))

In [ ]:
genet.save("gen_sub_model")
srgan.save("srgan_model")

In [ ]:
disc_hr.save("disc_hr_model")
disc_sr.save("disc_hr_model")

In [ ]:
import math
def compute_psnr(img1, img2):
  img1 = img1.astype(np.float64) / 255.
  img2 = img2.astype(np.float64) / 255.
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return "Same Image"
  return 10 * math.log10(1. / mse)
def psnr_test(test_LR,test_HR):
  image_ = test_LR
  image_gen_ = genet.predict(image_)
  
  psnr_result = compute_psnr(np.resize(image_gen_,(100,96,96,3)),test_HR)
  return psnr_result

In [ ]:
psnr_test(test_x[:100],test_y[:100])